# soyNLP 모듈
## h2로 쓰고싶네요
* 2016-10-20 뉴스 다운로드
* 다운 받는데 시간이 `오래걸림`.
```
pip install soynlp
```
jupyter-notebook 단축기
* `a` : 위에 셀 추가
* `b` : 아래에 셀 추가
* `dd` : 셀 삭제
* `m` : 문서 셀로 변경
* `m` : 문서 셀로 변경
* `y` : 코드 셀로 변경

In [9]:
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

('2016-10-20.txt', <http.client.HTTPMessage at 0x1e5b54d5820>)

In [3]:
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

corpus = DoublespaceLineCorpus("2016-10-20.txt")
len(corpus)

30091

In [4]:
i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    print('\n')
    i = i+1
  if i == 3:
    break

19  1990  52 1 22


오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성

In [21]:
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 0.788 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [20]:
from soynlp.tokenizer import MaxScoreTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

NameError: name 'word_score_table' is not defined

-0.0

#Noun Extractor ver 2

In [22]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

corpus_path = 'test.txt'
sents = DoublespaceLineCorpus(corpus_path, iter_sent=True)

noun_extractor = LRNounExtractor_v2(verbose=True)
nouns = noun_extractor.train_extract(sents)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 671 from 1 sents. mem=0.829 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=861, mem=0.829 Gb
[Noun Extractor] batch prediction was completed for 311 words
[Noun Extractor] checked compounds. discovered 0 compounds
[Noun Extractor] postprocessing detaching_features : 87 -> 87
[Noun Extractor] postprocessing ignore_features : 87 -> 84
[Noun Extractor] postprocessing ignore_NJ : 84 -> 84
[Noun Extractor] 84 nouns (0 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.829 Gb                    
[Noun Extractor] 34.03 % eojeols are covered


In [28]:
from pprint import pprint as pp

pp(nouns.keys())

dict_keys(['1950년', '있다.”-', '2020년', '중소기업', '경제성장', '서울신문', '저소득층', '노동조합', '무역전쟁', '불평등’', '세계경제', '장기적', '시스템', '중산층', '정치적', '대기업', '대통령', '근로자', '양극화', '90%', '증가분', '행정부', '일자리', '민주화', '인터뷰', '사다리', '노동자', '법인세', '자신들', '세계적', '10%', '구조', '정책', '극심', '심각', '이상', '이윤', '미국', '상황', '기업', '계층', '중국', '경제', '나라', '교수', '사회', '1%', '‘부', '노조', '증세', '교육', '자신', '임금', '집중', '30', '우려', '투자', '중반', '연구', '추진', '감세', '효과', '결과', '세계', '고용', '위험', '독점', '약화', '위협', '잘못', '시작', '10', '증가', '대', '통', '강', '것', '반', '심', '편', '등', '점', '때', '장'])


In [72]:
file = open("한국어불용어100.txt", 'r', encoding="utf-8")
stopword=[]

while True:
    line = file.readline()
    if not line: break
    wordlist = line.split('\t')
    if (wordlist[1].startswith('N')):
        stopword.append(wordlist[0])

print(stopword)

['것', '수', '이', '나', '사람', '등', '우리', '때', '년', '말', '일', '때문', '그것', '일', '문제', '사회', '중', '씨', '지금', '속', '하나', '집', '월', '데', '자신', '내', '경우', '명', '생각', '시간', '그녀', '앞', '번', '여자', '개', '전', '사실', '점', '정도', '원', '소리']


In [73]:
result = []
for w in nouns.keys():
    if w not in stopword:
        result.append(w)

print(nouns.keys())
print(result)

dict_keys(['1950년', '있다.”-', '2020년', '중소기업', '경제성장', '서울신문', '저소득층', '노동조합', '무역전쟁', '불평등’', '세계경제', '장기적', '시스템', '중산층', '정치적', '대기업', '대통령', '근로자', '양극화', '90%', '증가분', '행정부', '일자리', '민주화', '인터뷰', '사다리', '노동자', '법인세', '자신들', '세계적', '10%', '구조', '정책', '극심', '심각', '이상', '이윤', '미국', '상황', '기업', '계층', '중국', '경제', '나라', '교수', '사회', '1%', '‘부', '노조', '증세', '교육', '자신', '임금', '집중', '30', '우려', '투자', '중반', '연구', '추진', '감세', '효과', '결과', '세계', '고용', '위험', '독점', '약화', '위협', '잘못', '시작', '10', '증가', '대', '통', '강', '것', '반', '심', '편', '등', '점', '때', '장'])
['1950년', '있다.”-', '2020년', '중소기업', '경제성장', '서울신문', '저소득층', '노동조합', '무역전쟁', '불평등’', '세계경제', '장기적', '시스템', '중산층', '정치적', '대기업', '대통령', '근로자', '양극화', '90%', '증가분', '행정부', '일자리', '민주화', '인터뷰', '사다리', '노동자', '법인세', '자신들', '세계적', '10%', '구조', '정책', '극심', '심각', '이상', '이윤', '미국', '상황', '기업', '계층', '중국', '경제', '나라', '교수', '1%', '‘부', '노조', '증세', '교육', '임금', '집중', '30', '우려', '투자', '중반', '연구', '추진', '감세', '효과', '결과', '세계', '고용', '위험', '독점', '약화', '위협', '잘

In [121]:
class ReaderExam:
    def __init__(self, path):
        self.path = path
    
    def read(self, idx):
        res = open(self.path,'r', encoding="UTF-8")
        return res.read()

reader = ReaderExam("test.txt")

In [120]:
import sys
import io
import re # 정규표현식 패키지
from kiwipiepy import Kiwi, Option
import tomotopy as tp



filename="test.txt"  #파일 경로
file = open(filename, 'r', encoding="UTF-8")
kiwi=Kiwi()
kiwi.extract_add_words(reader.read)
kiwi.prepare()
stopwords = set(stopword)



def tokenize(sent): # 파일의 라인을 분석할 tokenize 함수
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용한다, 분석할때 나오는 결과에서 단어만 추출
    return [word
            for word, tag, _, _ in res
            if tag.startswith('NN') and word not in stopwords] #불용어사전 적용
    print(res)

print(tokenize(file.read()))



NameError: name 'path' is not defined